In [ ]:
# parameter cell
DEVICES = "1,2"
FROM = "2023-01-01"
TO = "2024-01-01"
ORDER = 1  # will be pass to param order in biLouvian call

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = DEVICES

import re
import cupy as cp
import pandas as pd
from tqdm import tqdm
import cudf
import networkx as nx
from cuml.preprocessing import LabelEncoder

tqdm.pandas()

# Import preprocessed data with active lenders only, and filter by the Paramteres

In [ ]:
ds = cudf.read_parquet("../data/gen/preprocessed_2023-08-28T11-09-39_from_2019-01-01_activelender.parquet")
ds["sector_name"] = ds["sector_name"].astype("category")
ds["geocode_country_name"] = ds["geocode_country_name"].astype("category")
ds["activity_name"] = ds["activity_name"].astype("category")
ds.drop(columns=["tags"], inplace=True)

In [ ]:
ds.dropna(axis=0, how="all", inplace=True)
print(len(ds))
ds.tail(2)

In [ ]:
ds = ds[(ds.fundraisingDate >= FROM) & (ds.fundraisingDate < TO)]
"the number of Loans (might duplicated) under investigation is", len(ds)

In [ ]:
ds.drop_duplicates(inplace=True)

In [ ]:
ds.duplicated(subset=["project_id", "lender_id", "loan_shareAmount", "loan_date"]).sum()

# Stats

In [ ]:
"number of Projects", ds.project_id.unique().count()

In [ ]:
"number of Lenders", ds.lender_id.unique().count()

In [ ]:
"number of Loans", ds.loan_id.nunique()

# Contruct `Lender-Sector` bipartite graph

## Create edge list of the graph

In [ ]:
LS = ds.groupby(["lender_id", "lender_publicId", "sector_id", "sector_name"]).agg(
    {"sector_name": "first", "loan_shareAmount": "sum", "loan_date": "count"}
)
LS.reset_index(inplace=True)
LS.rename(columns={"loan_shareAmount": "loan_amount", "loan_date": "loan_count"}, inplace=True)
LS = LS[LS["loan_amount"] > 0]
LS

## Convert to the format that `biLouvian` can understand

In [ ]:
le1 = LabelEncoder()
LS["V1"] = le1.fit_transform(LS["sector_name"])
le2 = LabelEncoder()
LS["V2"] = le2.fit_transform(LS["lender_id"]) + LS["V1"].max() + 1
LS.tail(3)

In [ ]:
# total number of vertex
vertex_count = LS["lender_id"].nunique() + LS["sector_name"].nunique()
assert vertex_count == LS["V1"].nunique() + LS["V2"].nunique()
print(vertex_count)

In [ ]:
dictionary1 = LS[["V1", "sector_name"]].drop_duplicates()
dictionary1.rename(columns={"sector_name": "name", "V1": "id"}, inplace=True)
dictionary1["name"] = dictionary1["name"].astype(str)
dictionary1.sort_values(by=["id"]).tail(3)

In [ ]:
dictionary2 = LS[["V2", "lender_publicId"]].drop_duplicates()
dictionary2.rename(columns={"lender_publicId": "name", "V2": "id"}, inplace=True)
dictionary2.head(1)

In [ ]:
# concat two dictionaries
dictionary = cudf.concat([dictionary1, dictionary2])
print(len(dictionary))
assert len(dictionary) == vertex_count
dictionary.tail()

In [ ]:
prefix = f"checkpoints/LS_bipartite_active_from{FROM}_to{TO}_order{ORDER}"  # the text "bipartite" is mandatory
filename = f"{prefix}.csv"
dictfile = f"{prefix}_Dictionary.txt"  # "_Dictionary.txt" is mandatory
LS[["V1", "V2", "loan_amount"]].to_csv(filename, sep="\t", header=False, index=False)  # "\t" is mandatory
dictionary[["id", "name"]].to_csv(dictfile, sep="\t", header=False, index=False)

# Run `biLouvian`

Note: Have to build the `biLouvian` binary first. Refer to `doc/BUILD_BILOUVIAN.md`

In [ ]:
!../extra/biLouvian -i $filename -order $ORDER